### 背景建模
背景去除是在很多视觉应用里的主要预处理步骤。例如，摄像这样的场景。一个固定摄像头对顾客进行计数，或者交通摄像头对汽车信息进行提取。在所有这些情况下，首先你需要把人或者车辆单独提取出来。技术上你需要从静态背景里把移动的前景提取出来。

如果你有一个单独的背景的图像，比如没有顾客的房子的图像，或者没有汽车的路的图像，这就很简单了。值需要在新的图像里把背景去掉就行了。你得到的就是前景。但是在大多数情况下，你可能没有这种图像，所以我们需要从我们有的图像里把背景去掉。当车有影子的时候这个情况就很复杂了。因为影子也在动。简单的去除会把影子也作为前景。

背景建模的一般方法可分为：帧差、中值滤波、高斯平均、混合高斯、Vibe和以纹理为特征的背景建模等。
 
### 帧差法

由于场景中的目标在运动，目标的影像在不同图像帧中的位置不同。该类算法对时间上连续的两帧图像进行差分运算，不同帧对应的像素点相减，判断灰度差的绝对值，当绝对值超过一定阈值时，即可判断为运动目标，从而实现目标的检测功能。

帧差法非常简单，但是会引入噪音和空洞问题

### 混合高斯模型

在进行前景检测前，先对背景进行训练，对图像中每个背景采用一个混合高斯模型进行模拟，每个背景的混合高斯的个数可以自适应。然后在测试阶段，对新来的像素进行GMM匹配，如果该像素值能够匹配其中一个高斯，则认为是背景，否则认为是前景。由于整个过程GMM模型在不断更新学习中，所以对动态背景有一定的鲁棒性。最后通过对一个有树枝摇摆的动态背景进行前景检测，取得了较好的效果。

#### 混合高斯模型学习方法

- 1.首先初始化每个高斯模型矩阵参数。

- 2.取视频中T帧数据图像用来训练高斯混合模型。来了第一个像素之后用它来当做第一个高斯分布。

- 3.当后面来的像素值时，与前面已有的高斯的均值比较，如果该像素点的值与其模型均值差在3倍的方差内，则属于该分布，并对其进行参数更新。

- 4.如果下一次来的像素不满足当前高斯分布，用它来创建一个新的高斯分布。


#### 混合高斯模型测试方法

在测试阶段，对新来像素点的值与混合高斯模型中的每一个均值进行比较，如果其差值在2倍的方差之间的话，则认为是背景，否则认为是前景。将前景赋值为255，背景赋值为0。这样就形成了一副前景二值图。

#### BackgroundSubtractorMOG

这是一个高斯混合背景/前景分割算法。它使用一个方法通过K高斯分布的混合来给每个背景像素建模（K=3到5）。混合的权重表示那些颜色留在场景中的时间规模。可能的背景颜色是那些留的时间更长更静止的。

使用函数cv2.createBackgroundSubtractorMOG()创建一个背景对象.它有一些可选的参数，比如历史长度，高斯混合数量，阈值等。它们全都是默认值
然后在视频循环里，使用backgroundsubtractor.apply()方法来得到背景掩图

#### BackgroundSubtractorMOG2

这也是高斯混合模型的前景/背景分割算法，这个算法的一个重要特点是它为每个像素选择一个合适的高斯分布数。它提供了更好的由于光照变化因素导致场景变化的适应性。

BackgroundSubtractorMOG2参数定义如下：

   history-历史帧的长度

   varThreshold-变量的阈值

   bShadowDetection-是否进行阴影的检测这里，
   
   你可以选择是否检测影子。bShadowDetection = True（默认的），它会检测并标记影子，但是会减低速度，Shadows会被标记成灰色。

##### BackgroundSubtractorGMG

这个算法合并了统计背景图像判断和每像素贝叶斯分割。

它使用最开始的一些帧（默认120）来对背景建模。它使用概率前景分割算法通过贝叶斯推论识别可能的前景物体。判断是能适应的。最新发现的权重更好，以适应光线变化。一些形态学过滤运算比如开和闭被使用来去掉不需要的噪点。你会在最开始的一些帧里得到一个黑色窗口

最好是通过形态学的开运算对结果使用，来移除噪点。

In [2]:
import numpy as np
import cv2

#经典的测试视频
cap = cv2.VideoCapture('img/test.avi')
#形态学操作需要使用
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
#创建混合高斯模型用于背景建模
fgbg = cv2.createBackgroundSubtractorMOG2()

while(True):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    #形态学开运算去噪点
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    #寻找视频中的轮廓
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        #计算各轮廓的周长
        perimeter = cv2.arcLength(c,True)
        if perimeter > 188:
            #找到一个直矩形（不会旋转）
            x,y,w,h = cv2.boundingRect(c)
            #画出这个矩形
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)    

    cv2.imshow('frame',frame)
    cv2.imshow('fgmask', fgmask)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


### 光流估计

光流是空间运动物体在观测成像平面上的像素运动的“瞬时速度”，根据各个像素点的速度矢量特征，可以对图像进行动态分析，例如目标跟踪。

- 亮度恒定：同一点随着时间的变化，其亮度不会发生改变。

- 小运动：随着时间的变化不会引起位置的剧烈变化，只有小运动情况下才能用前后帧之间单位位置变化引起的灰度变化去近似灰度对位置的偏导数。

- 空间一致：一个场景上邻近的点投影到图像上也是邻近点，且邻近点速度一致。因为光流法基本方程约束只有一个，而要求x，y方向的速度，有两个未知变量。所以需要连立n多个方程求解。

#### 光流：
光流的概念是指在连续的两帧图像当中，由于图像中的物体移动或者摄像头的移动而使得图像中的目标的运动叫做光流。（说简单点，考虑摄像头不会动的情况，就是一个视频当中有一个运动目标，那么这个视频中的相邻两帧中运动的目标就是光流）

#### 光流有很多应用场景如下：

    运动恢复结构
    视频压缩
    视频防抖动
    等等
    
#### Lucas-Kanade算法：    
https://blog.csdn.net/tengfei461807914/article/details/80978947

#### cv2.calcOpticalFlowPyrLK():
参数：
- prevImage 前一帧图像

- nextImage 当前帧图像

- prevPts 待跟踪的特征点向量

- winSize 搜索窗口的大小

- maxLevel 最大的金字塔层数

返回：

- nextPts 输出跟踪特征点向量

- status 特征点是否找到，找到的状态为1，未找到的状态为0

大致流程就是，首先获取视频或者摄像头的第一帧图像。用goodFeaturesToTrack函数获取初始化的角点，然后开始无限循环获取视频图像帧，将新图像和上一帧图像放入calcOpticalFlowPyrLK函数当中，从而获取新图像的光流。

In [3]:
import numpy as np
import cv2

cap = cv2.VideoCapture('img/test.avi')
#cap = cv2.VideoCapture(0)
# 角点检测所需参数
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7)

# lucas kanade参数
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2)

# 随机颜色条
color = np.random.randint(0,255,(100,3))

# 拿到第一帧图像
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# 返回所有检测特征点，需要输入图像，角点最大数量（效率），品质因子（特征值越大的越好，来筛选）
# 距离相当于这区间有比这个角点强的，就不要这个弱的了
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# 创建一个mask
mask = np.zeros_like(old_frame)

while(True):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 需要传入前一帧和当前图像以及前一帧检测到的角点
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # st=1表示
    good_new = p1[st==1]
    good_old = p0[st==1]

    # 绘制轨迹
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break

    # 更新
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

OpenCV中的稠密光流：

LK算法计算的是稀疏的特征点光流，如样例当中计算的是使用 Shi-Tomasi算法得到的特征点。opencv当总提供了查找稠密光流的方法。该方法计算一帧图像当中的所有点。该方法是基于Gunner Farneback提出的一篇论文Two-Frame Motion Estimation Based on Polynomial Expansion。

flow=cv.calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags)

返回值是每个像素点的位移

参数

    prev 输入8位单通道图片
    next 下一帧图片，格式与prev相同
    flow 与返回值相同，得到一个CV_32FC2格式的光流图，与prev大小相同
    pyr_scale 构建图像金字塔尺度
    levels 图像金字塔层数
    winsize 窗口尺寸，值越大探测高速运动的物体越容易，但是越模糊，同时对噪声的容错性越强
    iterations 对每层金字塔的迭代次数
    poly_n 每个像素中找到多项式展开的邻域像素的大小。越大越光滑，也越稳定
    poly_sigma 高斯标准差，用来平滑倒数
    flags 光流的方式，有OPTFLOW_USE_INITIAL_FLOW 和OPTFLOW_FARNEBACK_GAUSSIAN 两种
